In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as D
import tables as tb
from sklearn.metrics import (matthews_corrcoef, 
                             confusion_matrix, 
                             f1_score, 
                             roc_auc_score,
                             accuracy_score,
                             roc_auc_score)

In [2]:
# set the device to GPU if available
device = torch.device('cpu')

In [4]:
MAIN_PATH = '.'
DATA_FILE = 'mt_data.h5'
MODEL_FILE = 'chembl_mt.model'
N_WORKERS = 8 # Dataloader workers, prefetch data in parallel to have it ready for the model after each batch train
BATCH_SIZE = 32 # https://twitter.com/ylecun/status/989610208497360896?lang=es
LR = 2 # Learning rate. Big value because of the way we are weighting the targets
N_EPOCHS = 8 # You should train longer!!!


In [5]:
class ChEMBLDataset(D.Dataset):
    
    def __init__(self, file_path):
        self.file_path = file_path
        with tb.open_file(self.file_path, mode='r') as t_file:
            self.length = t_file.root.fps.shape[0]
            self.n_targets = t_file.root.labels.shape[1]
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        with tb.open_file(self.file_path, mode='r') as t_file:
            structure = t_file.root.fps[index]
            labels = t_file.root.labels[index]
        return structure, labels


dataset = ChEMBLDataset(f"{MAIN_PATH}/{DATA_FILE}")
validation_split = .2
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

train_sampler = D.sampler.SubsetRandomSampler(train_indices)
test_sampler = D.sampler.SubsetRandomSampler(test_indices)

# dataloaders can prefetch the next batch if using n workers while
# the model is tranining
train_loader = torch.utils.data.DataLoader(dataset,
                                           batch_size=BATCH_SIZE,
                                           num_workers=N_WORKERS,
                                           sampler=train_sampler)

test_loader = torch.utils.data.DataLoader(dataset, 
                                          batch_size=BATCH_SIZE,
                                          num_workers=N_WORKERS,
                                          sampler=test_sampler)

In [6]:
class ChEMBLMultiTask(nn.Module):
    """
    Architecture borrowed from: https://arxiv.org/abs/1502.02072
    """
    def __init__(self, n_tasks):
        super(ChEMBLMultiTask, self).__init__()
        self.n_tasks = n_tasks
        self.fc1 = nn.Linear(1024, 2000)
        self.fc2 = nn.Linear(2000, 100)
        self.fc3 = nn.Linear(2000, 100)
        self.dropout = nn.Dropout(0.25)

        # add an independet output for each task int the output laer
        for n_m in range(self.n_tasks):
            self.add_module(f"y{n_m}o", nn.Linear(100, 1))
        
    def forward(self, x):
        h1 = self.dropout(F.relu(self.fc1(x)))
        h2 = F.relu(self.fc2(h1))
        out = [torch.sigmoid(getattr(self, f"y{n_m}o")(h2)) for n_m in range(self.n_tasks)]
        return out
    
# create the model, to GPU if available
model = ChEMBLMultiTask(dataset.n_targets).to(device)

# binary cross entropy
# each task loss is weighted inversely proportional to its number of datapoints, borrowed from:
# http://www.bioinf.at/publications/2014/NIPS2014a.pdf
with tb.open_file(f"{MAIN_PATH}/{DATA_FILE}", mode='r') as t_file:
    weights = torch.tensor(t_file.root.weights[:])
    weights = weights.to(device)

criterion = [nn.BCELoss(weight=w) for x, w in zip(range(dataset.n_targets), weights.float())]

# stochastic gradient descend as an optimiser
optimizer = torch.optim.SGD(model.parameters(), LR)

In [ ]:
# model is by default in train mode. Training can be resumed after .eval() but needs to be set to .train() again
model.train()
for ep in range(N_EPOCHS):
    for i, (fps, labels) in enumerate(train_loader):
        # move it to GPU if available
        fps, labels = fps.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(fps)
        
        # calc the loss
        loss = torch.tensor(0.0).to(device)
        for j, crit in enumerate(criterion):
            # mask keeping labeled molecules for each task
            mask = labels[:, j] >= 0.0
            if len(labels[:, j][mask]) != 0:
                # the loss is the sum of each task/target loss.asi
                # there are labeled samples for this task, so we add it's loss
                loss += crit(outputs[j][mask], labels[:, j][mask].view(-1, 1))

        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            print(f"Epoch: [{ep+1}/{N_EPOCHS}], Step: [{i+1}/{len(train_indices)//BATCH_SIZE}], Loss: {loss.item()}")

Epoch: [1/8], Step: [500/17789], Loss: 0.016702646389603615


In [109]:
from sklearn.externals import joblib

In [110]:
torch.save(model.state_dict(), f"./{MODEL_FILE}")

In [111]:
model = ChEMBLMultiTask(560) # number of tasks
model.load_state_dict(torch.load(f"./{MODEL_FILE}"))
model.eval()

ChEMBLMultiTask(
  (fc1): Linear(in_features=1024, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=100, bias=True)
  (dropout): Dropout(p=0.25)
  (y0o): Linear(in_features=100, out_features=1, bias=True)
  (y1o): Linear(in_features=100, out_features=1, bias=True)
  (y2o): Linear(in_features=100, out_features=1, bias=True)
  (y3o): Linear(in_features=100, out_features=1, bias=True)
  (y4o): Linear(in_features=100, out_features=1, bias=True)
  (y5o): Linear(in_features=100, out_features=1, bias=True)
  (y6o): Linear(in_features=100, out_features=1, bias=True)
  (y7o): Linear(in_features=100, out_features=1, bias=True)
  (y8o): Linear(in_features=100, out_features=1, bias=True)
  (y9o): Linear(in_features=100, out_features=1, bias=True)
  (y10o): Linear(in_features=100, out_features=1, bias=True)
  (y11o): Linear(in_features=100, out_features=1, bias=True)
  (y12o): Linear(in_features=100, out_features=1, bias=True)
  (y13o): Linear(in_features=100, out_featur

In [112]:
from rdkit import Chem
from rdkit.Chem import AllChem
a="CC(C)NCC(COC1=CC=C(C=C1)CCOC)O"

In [113]:
m=Chem.MolFromSmiles(a)

In [114]:
j=AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)

In [29]:
def sigmoid(z):
    return [1 / (1 + math.exp(-n)) for n in z]

In [35]:
from rdkit import Chem, DataStructs
np_fps = []
arr = np.zeros((1,))
DataStructs.ConvertToNumpyArray(j, arr)
np_fps.append(arr)


In [38]:
import math
def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]

In [42]:
def calc_fp(smiles, fp_size, radius):
    """
    calcs morgan fingerprints as a numpy array.
    """
    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    mol.UpdatePropertyCache(False)
    Chem.GetSSSR(mol)
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius, nBits=fp_size)
    a = np.zeros((0,), dtype=np.float32)
    Chem.DataStructs.ConvertToNumpyArray(fp, a)
    return a

In [1]:
FP_SIZE = 1024
RADIUS = 2
from rdkit.Chem import rdMolDescriptors

In [2]:
j=calc_fp("COCCC1=CC=C(OCC(O)CNC(C)C)C=C1", 1024, 2)

NameError: name 'calc_fp' is not defined

In [130]:
a=torch.from_numpy(j)

In [131]:
model(a)

[tensor([0.2927], grad_fn=<SigmoidBackward>),
 tensor([0.1845], grad_fn=<SigmoidBackward>),
 tensor([0.3247], grad_fn=<SigmoidBackward>),
 tensor([0.1918], grad_fn=<SigmoidBackward>),
 tensor([0.3395], grad_fn=<SigmoidBackward>),
 tensor([0.5209], grad_fn=<SigmoidBackward>),
 tensor([0.5722], grad_fn=<SigmoidBackward>),
 tensor([0.2395], grad_fn=<SigmoidBackward>),
 tensor([0.0893], grad_fn=<SigmoidBackward>),
 tensor([0.0521], grad_fn=<SigmoidBackward>),
 tensor([0.0689], grad_fn=<SigmoidBackward>),
 tensor([0.2498], grad_fn=<SigmoidBackward>),
 tensor([0.2792], grad_fn=<SigmoidBackward>),
 tensor([0.0671], grad_fn=<SigmoidBackward>),
 tensor([0.3126], grad_fn=<SigmoidBackward>),
 tensor([0.5663], grad_fn=<SigmoidBackward>),
 tensor([0.0680], grad_fn=<SigmoidBackward>),
 tensor([0.4284], grad_fn=<SigmoidBackward>),
 tensor([0.7889], grad_fn=<SigmoidBackward>),
 tensor([0.0668], grad_fn=<SigmoidBackward>),
 tensor([0.1940], grad_fn=<SigmoidBackward>),
 tensor([0.2480], grad_fn=<Sigmoid